In [1]:
from __future__ import division 
import re, math, random 
import matplotlib.pyplot as plt 
from collections import defaultdict, Counter
from functools import partial, reduce

import numpy as np

In [2]:
def vector_add(v, w):
    """adds two vector componentwise """
    return [v_i + w_i for v_i, w_i in zip(v,w)]

In [3]:
v = [x for x in range(1, 11, 2)]
w = [y for y in range(11, 21, 2)]

In [4]:
vector_add(v, w)

# [v for v in range(1, 11, 2)] == [1, 3, 5, 7, 9]
# [w for w in range(10, 21, 2)] == [11, 13, 15, 17, 19]

[12, 16, 20, 24, 28]

In [6]:
# numpy version
np.array(v) + np.array(w)

array([12, 16, 20, 24, 28])

In [7]:
# vector, numpy 덧셈의 속도 비교
%timeit vector_add(v, w)
%timeit np.array(v) + np.array(w)

603 ns ± 0.971 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
2.06 µs ± 8.12 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
def vector_subtract(v, w):
    """subtracts two vectors componentwise"""
    return [v_i - w_i for v_i, w_i in zip(v,w)]

In [9]:
vector_subtract(v, w)

[-10, -10, -10, -10, -10]

In [10]:
# numpy version
np.array(v) - np.array(w)

array([-10, -10, -10, -10, -10])

##### 여러 개의 vector를 list로 만들어서 sum 연산을 하는 경우

In [11]:
def vector_sum(vectors):
    return reduce(vector_add, vectors)

In [12]:
vectors = [v, w, v, w, v, w]
vector_sum(vectors)

[36, 48, 60, 72, 84]

In [13]:
def vector_sum_modified(vectors):
    return [sum(value) for value in zip(*vectors)]

In [14]:
vector_sum_modified(vectors)

[36, 48, 60, 72, 84]

In [15]:
%timeit vector_sum(vectors)
%timeit vector_sum_modified(vectors)

2.99 µs ± 18.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.11 µs ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
# numpy version
np.sum([v, w, v, w, v, w], axis=0)

# axis=0은 row [v, w, v, w, v, w]를 하나의 matrix로 생각했을 때, column별 sum operation
# axis=1은 row[v, w, v, w, v, w]를 하나의 maxtrix로 생각했을 때, row별 sum operation

array([36, 48, 60, 72, 84])

In [18]:
# np.sum([v, w, v, w, v, w], axis=1)

##### Scalar Vector의 연산
2[1, 2, 3, 4] = [2, 4, 6, 8]

In [21]:
def scalar_multiply(c, v):
    return [c * v_i for v_i in v]

In [22]:
v = [5, 6, 7, 8]
scalar = 3

scalar_multiply(scalar, v)

[15, 18, 21, 24]

In [23]:
# numpy version: numpy는 배열의 크기가 다르더라도 기본적인 vector연산을 가능하도록 지원한다. (broadcasting)

scalar * np.array(v)

array([15, 18, 21, 24])

##### vector의 평균 구하기
크기가 같은 vector를 matrix 형태로 입력했을 경우, 각 row별 평균을 구함

In [24]:
def vector_mean(vectors):
    """compute the vector whose i-th element is the mean of the
    i-th elements of the input vectors"""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

In [25]:
v = [1, 2, 3, 4]
w = [-4, -3, -2, -1]

vector_mean([v, v, v, v])

[1.0, 2.0, 3.0, 4.0]

In [26]:
np.mean([v, v, v, v], axis=0)

# axis=0은 row[v, w, v, w, v, w]를 하나의 matrix로 생각했을 때, column별로 mean operation
# axis=1은 row[v, w, v, w, v, w]를 하나의 maxtrix로 생각했을 때, row별로 mean operation

array([1., 2., 3., 4.])

##### Vector dot product
크기가 같은 두 개의 vector가 있으면, 같은 column에 해당하는 값을 곱한 후 모든 값을 더한다

In [27]:
def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

In [28]:
v = [1, 2, 3, 4]
w = [-4, -3, -2, -1]

dot(v, w)

-20

In [29]:
# numpy version
np.dot(v, w)

-20

##### 하나의 vector에서 값 element들을 제곱하여 더한 후 값을 반환

In [30]:
def sum_of_squares(v):
    """v_1 * v_1 + ... + v_n * v_n"""
    return dot(v, v)

In [32]:
v = [1, 2, 3, 4]
sum_of_squares(v) # v * v = [1, 4, 9, 16]

30

In [33]:
# numpy version
np.dot(v, v) # or sum(np.square(v))

30

##### magnitude 
하나의 vector를 dot_product한 후 양의 제곱근을 구한다